In [1]:
import numpy as np
from scipy.optimize import linprog
from scipy.optimize import minimize
import pandas as pd

# my functions

In [2]:
def my_grad(f, x, h=1e-6):
  grad = np.zeros_like(x)
  for i in range(len(x)):
    x_plus_h = x.copy()
    x_plus_h[i] += h
    # grad[i] = (f(x_plus_h) - f(x)) / h
    grad[i] = (f([(x[j] if j!=i else x[j] + h) for j in range(len(x))]) - f(x)) / h

  return grad

In [3]:
def norm(arr):
    return sum(i**2 for i in arr)**0.5

In [4]:
def find_xk(f, a,b, tol=1e-6, max_iter = 100):
    hk = 2 - (1 + 5**0.5) / 2

    x1 = a + hk * (b - a)        # начальные точки
    x2 = b - hk * (b - a)
    # print(x1,x2)
    f1 = f(x1)
    f2 = f(x2)
    # print(f1,f2)

    for _ in range(max_iter):
        # print(x1,x2)
        if norm(b - a) < tol:
            break
        # print("f1f2",f1,f2)
        if f1 < f2:
            b, x2, f2 = x2, x1, f1
            x1 = a + hk * (b - a)
            f1 = f(x1)
        else:
            a, x1, f1 = x1, x2, f2
            x2 = b - hk * (b - a)
            f2 = f(x2)

    return (a + b) / 2


# Frank-Wolf

In [5]:
def frank_wolf(fi, x0, A, b, A_eq, b_eq, max_iter, eps, view=0):
   xk = x0
   k = 0
   opt = {'k': 0,
          'x': x0,
          'f': 0,
          'status': -1}

   while (k <= max_iter):
      df = my_grad(fi, xk)
      if view: print(f'grad(x{k}) = {df}')

      # minimize
      if A is None:
          res = linprog(df, A_eq=A_eq, b_eq=b_eq)
      else:
          res = linprog(df, A_ub=-A, b_ub=-b, A_eq=A_eq, b_eq=b_eq)
      yk = res.x
      if yk is None:
          opt['status'] = -1
          if view: print("yk is None: ", res)
          return opt
      if res.status != 0:
          opt['status'] = -1
          if view: print("status of linprog is not 0: ", res)
          return opt
      if view: print(f'y{k} = {yk}')

      # alpha search for xk+1 = xk + ak(yk-xk) for 0<=ak<=1
      prev = xk
      xk = find_xk(fi,xk,yk)
      if view: print(f'x{k} = {xk}')

      # fill opt
      opt['k'] = k
      opt['f'] = fi(xk)
      opt['x'] = xk
      k += 1
      # break
      if (norm(xk - prev) < eps) and (abs(fi(xk) - fi(prev)) < eps):
          opt['status'] = 0
          break
   return opt

# example

f(x) -> min

A * x >= b

A_eq * x = b_eq

In [6]:
res_fw = []
res_min = []

## ex1

In [7]:
def f(x):
    return x[0]**0.25 + (x[1]/x[0])**0.25 + (64/x[1])**0.25
x0 = np.array([2,10])
A = np.array([[1, 0],
              [-1, 1],
              [0, -1]])
b = np.array([1, 0, -64])
A_eq, b_eq = None, None
max_iter = 100
eps = 0.001
opt = frank_wolf(f, x0, A, b, A_eq, b_eq, max_iter, eps, 1)
# opt
res = minimize(
    f,
    x0,
    constraints = {"type": "ineq", "fun": lambda x: A @ x - b},
    options={"maxiter": max_iter}, # Дополнительные параметры
)
# print(res)

grad(x0) = [0 0]
y0 = [ 1. 64.]
x0 = [ 1.98547162 10.78453228]
grad(x1) = [-0.04275941 -0.00079184]
y1 = [64. 64.]
x1 = [ 3.73024015 12.28174069]
grad(x2) = [ 0.00286183 -0.00333502]
y2 = [ 1. 64.]
x2 = [ 3.59077851 14.92352776]
grad(x3) = [-0.00356779 -0.00018834]
y3 = [64. 64.]
x3 = [ 3.93470002 15.20292971]
grad(x4) = [ 0.00040582 -0.00049953]
y4 = [ 1. 64.]
x4 = [ 3.9023656  15.74057396]
grad(x5) = [-7.47633067e-04 -4.49640325e-05]
y5 = [64. 64.]
x5 = [ 3.98389807 15.80604592]
grad(x6) = [ 9.14734954e-05 -1.13904441e-04]
y6 = [ 1. 64.]
x6 = [ 3.97586987 15.93571212]
grad(x7) = [-1.79431581e-04 -1.11084475e-05]
y7 = [64. 64.]
x7 = [ 3.99598901 15.9518225 ]
grad(x8) = [ 2.23190355e-05 -2.78630452e-05]
y8 = [ 1. 64.]
x8 = [ 3.99398482 15.98396454]
grad(x9) = [-4.44080328e-05 -2.76845213e-06]
y9 = [64. 64.]
x9 = [ 3.99899819 15.98797617]
grad(x10) = [ 5.54845059e-06 -6.92690350e-06]
y10 = [ 1. 64.]
x10 = [ 3.99849732 15.99599467]
grad(x11) = [-1.10720322e-05 -6.90114632e-07]
y11 = [64.

In [8]:
res_fw.append(opt)
res_min.append(res)

## ex2

In [9]:
def f2(x):
    return (x[0]-1)**2 + 2*(x[1]-1)**2 - 3
x0 = np.array([0,0])
A = np.array([[-1, -2],
              [-2, 1],
              [0, 1],
              [1, 0]])
b = np.array([-8, -12, 0, 0])
A_eq, b_eq = None, None
max_iter = 100
eps = 0.001
opt = frank_wolf(f2, x0, A, b, A_eq, b_eq, max_iter, eps, 1)
print(opt)
res = minimize(
    f2,
    x0,
    constraints = {"type": "ineq", "fun": lambda x: A @ x - b},
    options={"maxiter": max_iter}, # Дополнительные параметры
)
# print(res)

grad(x0) = [-1 -3]
y0 = [0. 4.]
x0 = [0.         0.99999987]
grad(x1) = [-1.99999900e+00  1.49236179e-06]
y1 = [6.4 0.8]
x1 = [0.99805061 0.96881081]
grad(x2) = [-0.00389779 -0.12475475]
y2 = [0. 4.]
x2 = [0.98840948 0.99809197]
grad(x3) = [-0.02318004 -0.00763013]
y3 = [6.4 0.8]
x3 = [0.99982981 0.99767393]
grad(x4) = [-0.00033938 -0.0093023 ]
y4 = [0. 4.]
x4 = [0.99910481 0.999851  ]
grad(x5) = [-0.00178939 -0.000594  ]
y5 = [6.4 0.8]
x5 = [0.99998668 0.99981837]
{'k': 5, 'x': array([0.99998668, 0.99981837]), 'f': np.float64(-2.9999999338417465), 'status': 0}


In [10]:
res_fw.append(opt)
res_min.append(res)

## ex3

In [11]:
def f2(x):
    return -(-x[0]**2 + x[0]*x[1] - 2 * x[1]**2 + 4*x[0]+6*x[1])
x0 = np.array([3,1])
A = np.array([[-1, -1],
              [1, 2],
              [0, 1],
              [1, 0]])
b = np.array([-4, 2, 0, 0])
A_eq, b_eq = None, None
max_iter = 1000
eps = 0.0001
opt = frank_wolf(f2, x0, A, b, A_eq, b_eq, max_iter, eps, 1)
print(opt)
res = minimize(
    f2,
    x0,
    constraints = {"type": "ineq", "fun": lambda x: A @ x - b},
    options={"maxiter": max_iter}, # Дополнительные параметры
)
# print(res)

grad(x0) = [ 1 -4]
y0 = [0. 4.]
x0 = [2.2500001 1.7499999]
grad(x1) = [-1.24999872 -1.24999848]
y1 = [4. 0.]
x1 = [2.25000039 1.74999961]
{'k': 1, 'x': array([2.25000039, 1.74999961]), 'f': np.float64(-12.249999999999407), 'status': 0}


In [12]:
res_fw.append(opt)
res_min.append(res)

##ex4

In [13]:
def f2(x):
    return np.sin(x[0]) + np.cos(x[1]**2)
x0 = np.array([0.1,0.1])
A = np.array([[-1, -1],
              [0,1],
              [1,0],
              [2,5]])
b = np.array([-4,0,0,1])
A_eq, b_eq = None, None
max_iter = 1000
eps = 0.0001
opt = frank_wolf(f2, x0, A, b, A_eq, b_eq, max_iter, eps, 1)
print(opt)
res = minimize(
    f2,
    x0,
    constraints = {"type": "ineq", "fun": lambda x: A @ x - b},
    options={"maxiter": max_iter}, # Дополнительные параметры
)
# print(res)

grad(x0) = [ 0.99500412 -0.002     ]
y0 = [0. 4.]
x0 = [0.05706443 1.77448741]
grad(x1) = [0.99837224 0.02560454]
y1 = [0.  0.2]
x1 = [0.0568861  1.76956718]
grad(x2) = [ 0.99838239 -0.03617953]
y2 = [0. 4.]
x2 = [0.02370246 3.07065605]
grad(x3) = [0.9997191  0.02550895]
y3 = [0.  0.2]
x3 = [0.01297803 1.77179662]
grad(x4) = [ 0.99991578 -0.00824811]
y4 = [0. 4.]
x4 = [0.00541594 3.07013467]
grad(x5) = [0.99998533 0.00584555]
y5 = [0.  0.2]
x5 = [0.00296694 1.77230372]
grad(x6) = [ 0.9999956 -0.0018801]
y6 = [0. 4.]
x6 = [1.23859067e-03 3.07001551e+00]
grad(x7) = [0.99999923 0.00135299]
y7 = [0.  0.2]
x7 = [6.78597169e-04 1.77241973e+00]
grad(x8) = [ 9.99999769e-01 -4.22501256e-04]
y8 = [0. 4.]
x8 = [2.83313431e-04 3.06998815e+00]
grad(x9) = [9.99999960e-01 3.21330629e-04]
y9 = [0.  0.2]
x9 = [1.55225412e-04 1.77244607e+00]
grad(x10) = [ 9.99999988e-01 -9.14365250e-05]
y10 = [0. 4.]
x10 = [6.48076020e-05 3.06998199e+00]
grad(x11) = [9.99999998e-01 8.93615182e-05]
y11 = [0.  0.2]
x11 = 

In [14]:
res_fw.append(opt)
res_min.append(res)

# result table

##fun

In [37]:
def round_arr(arr, r=3):
    t = 10**r
    arr2 = []
    for i in range(len(arr)):
        arr2.append(int(arr[i]*t)/t)
    return arr2

In [22]:
def compare_results_df(res_fw, res_min, eps=1e-3, r=4):
    for i in range(len(res_fw)):
        fw = res_fw[i]
        mn = res_min[i]

        fun_comparison = abs(fw['f'] - mn.fun) < eps

        status_comparison = 0 if ((fw['status'] == mn.status) == 0) else -1

        x_comparison = "-"

        if fw['k'] == mn.nit:
            nit_comparison = "равны"
        elif fw['k'] > mn.nit:
            nit_comparison = "Ф-В хуже"
        else:
            nit_comparison = "Ф-В лучше"

        data = {
            "Метод Франка-Вульфа": [fw['f'], fw['status'], round_arr(fw['x']), fw['k']],
            "Минимизация": [mn.fun, mn.status, round_arr(mn.x), mn.nit],
            "Сравнение": [fun_comparison, status_comparison, x_comparison, nit_comparison]
        }

        df = pd.DataFrame(data, index=["F", "статус", "x", "Кол-во итераций"])

        print(f"\nПример {i+1}:")
        print(df)


##see

In [ ]:
compare_results_df(res_fw, res_min)